# The Leaky Integrate-and-Fire (LIF) neuron model

# <font color='green'> Objectives </font>

In this notebook, we will build up a leaky integrate-and-fire (LIF) neuron model and study its dynamics in response to various types of inputs. In particular, we are going to write a few lines of code to  

1) simulate the LIF neuron model

2) drive the LIF neuron with external inputs, such as DC current, Gaussian white noise, and Poisson spike trains, etc.

3) study how different inputs affect the  LIF neuron's output (firing rate and spike time irregularity)

Here will be specially emphasize on identifying conditions (input statistics) under which a neuron can spike at low firing rates and in a irregular manner. The reason for focusing on this is that in most cases neocortical neurons spike in an irregular manner (but not quite Poissonian).

Please execute the cell below to initialize the notebook environment.

In [1]:
import matplotlib.pyplot as plt    # import matplotlib
import numpy as np                 # import numpy
import time                        # import time
import ipywidgets as widgets       # interactive display
from scipy.stats import pearsonr   # import pearson correlation

fig_w, fig_h = 8, 6
my_fontsize = 18
my_params = {'axes.labelsize': my_fontsize,
          'axes.titlesize': my_fontsize,
          'figure.figsize': (fig_w, fig_h),
          'font.size': my_fontsize,
          'legend.fontsize': my_fontsize-4,
          'lines.markersize': 8.,
          'lines.linewidth': 2.,
          'xtick.labelsize': my_fontsize-2,
          'ytick.labelsize': my_fontsize-2}

plt.rcParams.update(my_params)
my_layout = widgets.Layout()


ModuleNotFoundError: No module named 'ipywidgets'

In [ ]:
#@title Helper functions
def plot_volt_trace(pars, v, sp):
    '''
    Plot trajetory of membrane potential for a single neuron
  
    Expects:
    pars   : parameter dictionary
    v      : volt trajetory
    sp     : spike train
  
    Returns:
    figure of the membrane potential trajetory for a single neuron
    '''

    V_th = pars['V_th']
    dt, range_t = pars['dt'], pars['range_t']
    if sp.size:
       sp_num = (sp/dt).astype(int)-1
       v[sp_num] += 10

    plt.plot(pars['range_t'], v, 'b')
    plt.axhline(V_th, 0, 1, color='k', ls='--')
    plt.xlabel('Time (ms)')
    plt.ylabel('V (mV)')


# <font color='green'> Leaky integrate-and-fire (LIF) neuron model</font>

One of the simplest mathematical model of a neuron is the Leaky integrate-and-fire (LIF) model. The basic idea of LIF neuron was proposed in 1907 by Louis Édouard Lapicque, long before we understood the electrophysiology of a neuron [see a translation of Lapicque's paper [link text](https://pubmed.ncbi.nlm.nih.gov/17968583/)].

The subthreshold membrane potential dynamics of a LIF neuron is described by

\begin{eqnarray}
C_m\frac{dV}{dt} = -g_L(V-V_L) + I,
\end{eqnarray}

where $C_m$ is the membrane capacitance, $V$ is the membrane potential, $g_L$ is the leak conductance, $V_L$ is the resting potential, and $I$ is the external input current. 

Dividing both sides of the above equation by $g_L$ gives

\begin{align}
\tau_m\frac{dV}{dt} = -(V-V_L) + \frac{I}{g_L}\,,\quad (1)
\end{align}

where the time constant $\tau_m$ is defined as $\tau_m=C/g_L$. Below we will use Eqn.(1) to simulate the LIF neuron dynamics. 

If $I$ is sufficiently strong such that $V$ reaches a certain threshold value $V_{\rm th}$, $V$ is reset to a reset potential $V_{\rm reset}<V_{\rm th}$ and voltage is clamped to $V_{reset}$ for $t_{ref}$ms mimicking the refractoriness of the neuron during an action potential, _i.e._,

\begin{eqnarray}
\mathrm{if}\quad V(t)\geq V_{\rm th}&:& V(t^+)=V_{\rm reset}
\end{eqnarray}

Thus, the LIF model captures the facts that a neuron
- performes spatial and temporal integration of synaptic inputs 
- generates a spike when the voltage reaches a certain threshold
- goes refractory during the action potential
- membrane is leaky

The LIF model assumes that spatial and temporal integration of inputs is linear. Also, membrane potential dynamics close to the spike threshold is much slower in LIF neurons than in real neurons. 


# <font color='green'> Numerical discretization with forward Euler scheme </font>

To numerically solve the membrane potential dynamics of the LIF neuron, we need to discretize the continuous dynamics of Eqn. (1) by using a time-grid of stepsize $\Delta t$. Note that, to ensure an accurate approximation to the solution of Eqn. (1), the time step $\Delta t$ needs to be small compared to all other time constants in the system (here is $\tau_m$). The smaller value $\Delta t$ takes, the better the approximation is. However, the total simulation time will become longer.

On the time-grid of of stepsize $\Delta t$, the dynamics of Eqn. (1) is discretized to be 

\begin{eqnarray}
\tau_m \frac{\Delta V_n}{\Delta t} &=& -(V_n-V_L) +\frac{I_n}{g_L},
\end{eqnarray}

where $V_n = V(n\Delta t)$, and $I_n=I(n\Delta t)$. 

Then the updating process of $V_{n+1}$ at time $t_{n+1}=(n+1)\Delta t$ based on $V_{n}$ can be formed as 

\begin{eqnarray}
V_{n+1} &=& V_n+ \Delta V_n \\
\Delta V_n &=&  \left(-(V_n-V_L) +\frac{I}{g_L}\right)\frac{\Delta t}{\tau_m} \qquad (2)
\end{eqnarray}

# <font color='green'> Python code to simulate the LIF neuron </font>

We now **write** a Python code to calculate the Eqn.(2) and simulate the LIF neuron dynamics.

The cell below initializes a dictionary that stores parameters of the LIF neuron model and the simulation scheme. 

In [ ]:
#@title Default value function: `default_pars( **kwargs)` 
def default_pars( **kwargs):
    pars = {}
    
    ### typical neuron parameters###
    pars['V_th']    = -55. # spike threshold [mV]
    pars['V_reset'] = -75. #reset potential [mV]
    pars['tau_m']   = 10. # membrane time constant [ms]
    pars['g_L']     = 10. #leak conductance [nS]
    pars['V_init']  = -65. # initial potential [mV]
    pars['E_L']     = -75. #leak reversal potential [mV]
    pars['tref']    = 2. # refractory time (ms)

    ### simulation parameters ###
    pars['T'] = 400. # Total duration of simulation [ms]
    pars['dt'] = .1  # Simulation time step [ms]
    
    ### external parameters if any ###
    for k in kwargs:
        pars[k] = kwargs[k]
    
    pars['range_t'] = np.arange(0, pars['T'], pars['dt']) # Vector of discretized time points [ms]
        
    return pars

The cell below defines the function to simulate the LIF neuron when receiving external current inputs.

In [ ]:
#@title Function `run_LIF(pars, I)` 

def run_LIF(pars, I):
    '''
    Simulate the LIF dynamics with external input current 
    
    Expects:
    pars       : parameter dictionary
    I          : input current [pA]. The injected current here can be a value or an array
    
    Returns:
    rec_v      : mebrane potential
    rec_sp     : spike times
    '''
    
    # Set parameters
    V_th, V_reset = pars['V_th'], pars['V_reset']   
    tau_m, g_L = pars['tau_m'], pars['g_L']        
    V_init, E_L = pars['V_init'], pars['E_L']       
    dt, range_t = pars['dt'], pars['range_t'] 
    Lt = range_t.size 
    tref = pars['tref']
    # Initialize voltage and current 
    v = np.zeros(Lt)
    v[0] = V_init
    I = I * np.ones(Lt) 
    tr = 0. # the count for refractory duration
    # simulate the LIF dynamics 
    rec_spikes = []               # record spike times
    for it in range(Lt-1):
      if tr >0: # check for refractoriness
            v[it] = V_reset
            tr = tr-1
      elif v[it] >= V_th:         #reset voltage and record spike event
            rec_spikes.append(it)
            v[it] = V_reset
            tr = tref/dt
      #calculate the increment of the membrane potential
      dv = (-(v[it]-E_L) + I[it]/g_L) * (dt/tau_m)
        
      #update the membrane potential
      v[it+1] = v[it] + dv
        
    rec_spikes = np.array(rec_spikes) * dt
        
    return v, rec_spikes

print(help(run_LIF))

# <font color='green'> Response of an LIF model to different types of input currents </font>
In the following we will inject direct current and white noise to study the response of an LIF neuron.

### Constant current 
Execute the following cell to run the LIF neuron when receiving a DC current of 300 pA, and see the voltage response of the LIF neuron.

In [ ]:
pars = default_pars()
v, rec_spikes = run_LIF(pars, I = 300.)
plt.plot(pars['range_t'], v, 'b')
plt.xlim(0, 100)
plt.xlabel('Time (ms)')
plt.ylabel('V (mV)');

In [ ]:
#@title LIF neuron Explorer for DC input
my_layout.width = '450px'
@widgets.interact(
    I_dc  = widgets.FloatSlider(200., min=100., max=300., step=5., layout=my_layout),
    tau_m = widgets.FloatSlider(10., min=2., max=20., step=2., layout=my_layout)
)

def diff_DC(I_dc=200., tau_m=10.):
  pars = default_pars(T=100.)
  pars['tau_m'] = tau_m
  v, sp = run_LIF(pars, I = I_dc)
  plt.figure()
  plot_volt_trace(pars, v, sp)
  plt.legend(['Membrane\npotential', r'Threshold V$_{\mathrm{th}}$'], loc=[1.05, 0.75])
  plt.show()
#_ = widgets.interact(diff_DC, I_dc = (100., 300., 5.))

### Gaussian white noise (GWN) current 

Neurons in vivo never receive direct current like inputs. Given the noisy nature of neuronal activity in vivo, neurons receive noisy time varying inputs.

We next investigate the neuronal response when the LIF neuron receives Gaussian white noise $\xi(t)$ with mean 

\begin{eqnarray}
E[\xi(t)]=\mu=0,
\end{eqnarray}

and autocovariance 
\begin{eqnarray}
E[\xi(t)\xi(t+\tau)]=\sigma_\xi^2 \delta(\tau)
\end{eqnarray}

Hint: In order to have correct statistics of the GWN approximation on a time grid with increment $\Delta t$, the noise amplitide $\sigma_\xi$ needs to be scaled by $1/\sqrt{\Delta t}$. 

The cell below defines gaussian white noise currents.

In [ ]:
def my_GWN(pars, mu, sig, myseed=False):
    '''
    Expects:
    pars       : parameter dictionary
    mu         : noise baseline (mean)
    sig        : noise amplitute (standard deviation)
    myseed     : random seed. int or boolean
                 the same seed will give the same random number sequence
    
    Returns:
    I          : Gaussian white noise input
    '''
    
    # Retrieve simulation parameters
    dt, range_t = pars['dt'], pars['range_t']
    Lt = range_t.size
    
    # set random seed  
    # you can fix the seed of the random number generator so that the results are reliable 
    # however, when you want to generate multiple realization make sure that you change 
    # the seed for each new realization
    if myseed:
        np.random.seed(seed=myseed) 
    else:
        np.random.seed()
        
    #generate GWN
    # we divide here by 1000 to convert units to sec.
    I =  mu + sig * np.random.randn(Lt) / np.sqrt(dt/1000.) 
    
    return I

Execute the cell below to generate the GWN current and plot it out.

In [ ]:
pars = default_pars(T=100.)
sig_gwn = 5.
mu_gwn  = 250.
I_GWN = my_GWN(pars, mu = mu_gwn, sig=sig_gwn, myseed=2020)
v, sp = run_LIF(pars, I=I_GWN)
plt.figure(figsize=(12, 4))
plt.subplot(1,2,1)
plt.plot(pars['range_t'][::3], I_GWN[::3], 'b')
plt.xlabel('Time (ms)')
plt.ylabel(r'$I_{GWN}$ (pA)')
plt.subplot(1,2,2)
plot_volt_trace(pars, v, sp)
plt.tight_layout()
plt.show()

In [ ]:
#@title LIF neuron Explorer for noisy input

my_layout.width = '450px'
@widgets.interact(
    mu_gwn  = widgets.FloatSlider(200., min=100., max=300., step=5., layout=my_layout),
    sig_gwn = widgets.FloatSlider(2.5, min=0., max=5., step=.5, layout=my_layout)
)


def diff_GWN_to_LIF(mu_gwn,sig_gwn):
    pars = default_pars(T=100.)
    I_GWN = my_GWN(pars, mu = mu_gwn, sig=sig_gwn)
    v, sp = run_LIF(pars, I =I_GWN)
    plt.figure(figsize=(12, 4))
    plt.subplot(1,2,1)
    plt.plot(pars['range_t'][::3], I_GWN[::3], 'b')
    plt.xlabel('Time (ms)')
    plt.ylabel(r'$I_{GWN}$ (pA)')
    plt.subplot(1,2,2)
    plot_volt_trace(pars, v, sp)
    plt.tight_layout()
    plt.show()

#_ = widgets.interact(diff_GWN_to_LIF, mu_gwn = (-100., 400., 5.),sig_gwn = (0, 5., 0.5))   

Execute the cell below to inject the GWN current to the LIF neuron and see its voltage response.

## <font color='blue'> Think </font>
- As we increase input (DC value) or GWN parameters the spike count changes. How much can we increase the spike count and what might be relationship between GWN mean/std or DC value and spike count? 

- We have seen above that when we inject DC the neuron spikes in a regular manner (clock like) and this regularity is reduced when GWN is injected. The question is how much irregular can we make the neurons spiking by changing the parameters of the GWN? 


#### Firing rate and spike time irregularity
When we plot the output firing rate as a function of GWN mean or DC value, it is called the input-output transfer function of the neuron (so simply F-I curve).

Spike regularity can be quanitfied as **coefficient of variance of the inter-spike-interval**:
\begin{align}
\text{CV}_{\text{ISI}} = \frac{std(\text{isi})}{mean(\text{isi})}
\end{align}

*isi* can be calculate by using np.diff(spike_time)

For completely irregular spiking (i.e. a Poisson Point process) $\textbf{CV}_{\textbf{ISI}} \textbf{= 1}$. And for a clock like process because *std(isi)* is zero $\textbf{CV}_{\textbf{ISI}} \textbf{= 0}$. 

### Task: Neuron F-I curve and spike time regularity
To address the above questions we will systematically vary the mean and standard deviation of the injected GWN and plot the F-I curve of the LIF neuron.

In [ ]:
#@title F-I Explorer for different `sig_gwn`

my_layout.width = '450px'
@widgets.interact(
    sig_gwn = widgets.FloatSlider(3.0, min=0., max=6., step=0.5, layout=my_layout)
)

def diff_std_affect_fI(sig_gwn):
    pars = default_pars(T=1000.)
    I_mean = np.arange(100.,400., 10.)
    spk_count = np.zeros(len(I_mean))
    spk_count_dc = np.zeros(len(I_mean))

    for idx in range(len(I_mean)):
        I_GWN = my_GWN(pars, mu=I_mean[idx], sig=sig_gwn, myseed=2020)
        v, rec_spikes = run_LIF(pars, I =I_GWN)
        v_dc, rec_sp_dc = run_LIF(pars, I =I_mean[idx])
        spk_count[idx] = len(rec_spikes)
        spk_count_dc[idx] = len(rec_sp_dc)
    
    # Plot the F-I curve i.e. Output firing rate as a function of input mean.
    plt.figure()
    plt.plot(I_mean, spk_count, 'k', label=r'$\sigma_{\mathrm{GWN}}=%.2f$' % sig_gwn)
    plt.plot(I_mean, spk_count_dc, 'k--', alpha=0.5, lw=4, dashes=(2,2), label='DC input')
    plt.ylabel('Spike count')
    plt.xlabel('Average injected current (pA)')
    plt.legend(loc='best')
    plt.show()
    
#_ = widgets.interact(diff_std_affect_fI,sig_gwn = (0, 6., 1.))

#### Lets plot the results.
We are interested in two variables. 
- Spike count
- Irregularity of spike pattern



In [ ]:
#remove solutions
pars = default_pars(T=1000.)
mu_gwn  = 250.

sig_gwn1 = 0.5
I_GWN = my_GWN(pars, mu = mu_gwn, sig=sig_gwn1, myseed=2020)
v, sp1 = run_LIF(pars, I=I_GWN)
isi1 = np.diff(sp1)
cv1 = isi1.std()/isi1.mean()

sig_gwn2 = 3.0
I_GWN = my_GWN(pars, mu = mu_gwn, sig=sig_gwn2, myseed=2020)
v, sp2 = run_LIF(pars, I=I_GWN)
isi2 = np.diff(sp2)
cv2 = isi2.std()/isi2.mean()

plt.figure(figsize=(11, 4))
my_bins = np.linspace(10, 30, 20)
plt.subplot(1, 2, 1)
plt.hist(isi1, bins=my_bins, color='b', alpha=0.5);
plt.xlabel('ISI (ms)')
plt.ylabel('number')
plt.title(r'$\sigma=$%.1f, CV$_{\mathrm{isi}}$=%.3f' % (sig_gwn1, cv1))

plt.subplot(1, 2, 2)
plt.hist(isi2, bins=my_bins, color='b', alpha=0.5);
plt.xlabel('ISI (ms)')
plt.ylabel('number')
plt.title(r'$\sigma=$%.1f, CV$_{\mathrm{isi}}$=%.3f' % (sig_gwn2, cv2))
plt.tight_layout()
plt.show()

In [ ]:
#@title Spike irregularity Explorer for different `sig_gwn`

my_layout.width = '450px'
@widgets.interact(
    sig_gwn = widgets.FloatSlider(0.0, min=0., max=5., step=0.5, layout=my_layout)
)


def diff_std_affect_fI(sig_gwn):
    pars = default_pars(T=1000.)
    I_mean = np.arange(100.,400.,20)
    spk_count = np.zeros(len(I_mean))
    cv_isi = np.empty(len(I_mean))

    for idx in range(len(I_mean)):
        I_GWN = my_GWN(pars, mu=I_mean[idx], sig=sig_gwn)
        v, rec_spikes = run_LIF(pars, I =I_GWN)
        spk_count[idx] = len(rec_spikes)
        if len(rec_spikes)>3:
          isi = np.diff(rec_spikes)
          cv_isi[idx] = np.std(isi)/np.mean(isi)
    
    # Plot the F-I curve i.e. Output firing rate as a function of input mean.
    plt.plot(I_mean[spk_count>5],cv_isi[spk_count>5], 'bo', alpha=0.5)
    plt.xlabel('Average injected current (pA)')
    plt.ylabel(r'Spike irregularity ($\mathrm{CV}_\mathrm{ISI}$)');
    plt.ylim(-0.1, 1.5)
    plt.show()

#_ = widgets.interact(diff_std_affect_fI,sig_gwn = (0, 5., 1.))

# <font color='salmon'> Try to answer the following </font>
- Does standard deviation of the injected current affects the F-I curve in any qualitative manner?
- Why there might be a relationship between spike count (or rate) and CV_ISI? 
- Why increasing the mean of GWN reduced the CV_ISI?

# <font color='green'>Simulate synaptic conductance dynamics </font>

Even GWN is a quite unphysiological to neurons. In vivo neuron receive spiking input which can be either excitatory (that drives the neuron towards the spike threshold) or inhibitory (that drives the neuron away from the spike threshold). 

The spikes arriving at the neuron result in opening of channels that result in change in the neuron conductance and eventual flow of current whose direction is determined by the type of ion flowing.

This effect of an incoming spike at the neuron can be modelled a transient change in the neuron's conductance. The conductance change is convered to current  as:
\begin{align}
I_{\rm syn}(t) = -g_{\rm syn}(t)(V(t)-V_{\rm syn})  \\
\end{align}

The reversal potential $V_{\rm syn}$ describes the direction of current flow and the excitatory or inhibitory nature of the synapse. 

In a neuronal network, the total synaptic input current $I$ is a sum of both excitatory and inhibitory inputs. Assuming the total excitatory (inhibitory) conductance received at time $t$ is $g_E(t)$ ($g_I(t)$), and its corresponding reversal potential is $V_E$ ($V_I$). Then the total synaptic current can be described as: 

\begin{align}
I_{\rm syn}(V(t),t) = -g_E(t) (V-V_E) - g_I(t) (V-V_I). \\
\end{align}

Accordingly, the membrane potential dynamics of the LIF neuron under synaptic current drive becomes

\begin{align}
\tau_m\frac{dV(t)}{dt} = -(V(t)-V_L) - \frac{g_E(t)}{g_L} (V(t)-V_E) - \frac{g_I(t)}{g_L} (V(t)-V_I) + \frac{I_{\rm inj}}{g_L}.\quad (3)
\end{align}

We will use Eqn. (3) to simulate the conductance-based LIF neuron model below.

### Model of synaptic conductance transients

The dynamics of $g_E$ and $g_I$ is determined by the gating variables of the synaptic channels. Each synaptic receptor channel has two states: either open or closed. When an input arrives, we denote $s$ as the fraction of open channels, and denote $T$ as the concentration of neurotransmitters. By denoting $\alpha_s$ and $\beta_s$ as the rate of transition between the two states, we have the change of $s$ described as 

$$
\frac{ds}{dt} = \alpha_s T(1-s) - \beta_s s. 
$$ 

If we model $T$ by a sequence of short pulses $\delta(t)$ at the input arrival times $t_k$, we can show that the gating variable dynamics are given by the following first order kinetics 

$$
\frac{ds}{dt} = \alpha_s \sum_k\delta(t-t_k)(1-s) -s/\tau_s, \quad (4)
$$
where $\tau_s=1/\beta_s$ is defined as the synaptic time constant.


Staring at Eqn. (4) above, we expect that, under high density of pulses (high input rate), the gating variable $s$ will increase towards its bound, which we set to be 1 here. And because of the factor $(1-s)$, each input pulse will become less and less efficient in driving $s$. This phenomenon is known as synaptic saturation.

In fact, each open channel needs to take $\sim \tau_{s}$ to go back to the closed state. For fast AMPA (E) and GABA (I) dynamics with small $\tau_{s}$, it is often reasonable to assume that $s\ll 1$ such that one can ontain the following approximation

$$
\frac{ds}{dt} = \alpha_s \sum_k \delta(t-t_k) -s/\tau_s. \quad (5)
$$

Eqn. (5) can be analytically solved, and the solution takes the form of a linear summation of exponential functions.


### Synaptic conductance dynamics

The synaptic condutance is often modeled as $g=\bar{g}s$, where $\bar{g}$ is the maximum condutance if all synaptic channels are open. Accordingly, from Eqn. (5), we can describe the dynamics of excitatory and inhibitory condutances as 

\begin{eqnarray}
\frac{dg_E(t)}{dt} &=& \bar{g}_E \sum_k \delta(t-t_k) -g_E(t)/\tau_E, \\
\frac{dg_I(t)}{dt} &=& \bar{g}_I \sum_k \delta(t-t_k) -g_I(t)/\tau_I. 
\end{eqnarray}

In the absence of any presynaptic pulse inputs, the excitatory and inhibitory synaptic conductances will decay to zero with time constant $\tau_E$ and $\tau_I$ respectively. Each presynaptic pulse input will increase the conductance $g_{\rm X} (t)$ by an amount $\bar{g}_{X}$ for ${\rm X} =E, I$. Therefore, using the forward Euler scheme, we can derive the updating rule for the excitatory and inhibitory condutances

\begin{eqnarray}
\Delta g_X^n &=& -\frac{g_X^n}{\tau_X}\Delta t + \bar{g}_{X}\cdot N_{X}^{n} \\
g_X^{n+1} &=& g_X^n + \Delta g_X^n \quad {\rm for\ X}=E,\ I \\
\end{eqnarray}

where $N_{X}^{n}$ is the total number of the input pluses from presynaptic neurons within the time window $[n\Delta t, (n+1)\Delta t)$.

### Generation of Poisson type spike trains

Mathematically a spike train is a Point Process. One of the simplest model of a sequence of presynaptic pulse inputs is the Poisson process. We know that given temporal integration and refractoriness, neurons cannot behave as a Poisson Process and Gamma Process gives a better approximation. 
Here however, we will assume that the incoming spike are following Poisson statistics. Question arises how to simulate a Poisson Process. The generation of the Poisson process can be realized by at least the following two ways:

- By definition, for a Poisson process with rate $\lambda$, the probability of finding one event in the time window with a sufficiently small length $\Delta t$ is $P(N = 1) = \lambda \Delta t$. Therefore, in each time window, we generate a uniformly distributed random variable $r \in [0,1]$ and generate a Poisson event when $r <\lambda \Delta t$. This method allows us to generate Poisson distributed spikes in an online manner.


- The interval $t_{k+1}-t_{k}$ between two Poisson events with rate $\lambda$ follows the exponential distribution, i.e., $P(t_{k+1}-t_{k}<t) = 1 - e^{\lambda t}$. Therefore, we only need to generate a set of exponentially distributed variables $\{s_k\}$ to obtain the timing of Poisson events $t_{k+1}=t_{k}+s_{k}$. In this method we need to generate all future spikes at once. 

Below we use the first method and check the statistical property of the time interval between two Poisson events.

In [ ]:
#@title `Poisson_generator(pars, rate, n, myseed=False)`
def Poisson_generator(pars, rate, n, myseed=False):
    
    '''
    Generates poisson trains
    Expects:
    pars       : parameter dictionary
    rate       : noise amplitute [Hz]
    n          : number of Poisson trains
    myseed     : random seed. int or boolean
    
    Returns:
    pre_spike_train : spike train matrix, ith row represents whether
                      there is a spike in ith spike train over time
                      (1 if spike, 0 otherwise)
    '''
    
    # Retrieve simulation parameters
    dt, range_t = pars['dt'], pars['range_t']
    Lt = range_t.size
    
    # set random seed
    if myseed:
        np.random.seed(seed=myseed) 
    else:
        np.random.seed()
    
    # generate uniformly distributed random variables
    u_rand = np.random.rand(n, Lt)
    
    # generate Poisson train
    poisson_train = 1. * (u_rand<rate*dt/1000.)
    
    return poisson_train

# the function plot the raster of the Poisson spike train
def my_raster_Poisson(range_t, spike_train, n):
    '''
    Generates poisson trains
    Expects:
    range_t     : time sequence 
    spike_train : binary spike trains, with shape (N, Lt)
    n           : number of Poisson trains plot
    
    Returns:
    Raster_plot of the spike train
    '''
    #find the number of all the spike trains
    N = spike_train.shape[0]

    # n should smaller than N:
    if n > N:
       print('The number n exceeds the size of spike trains')
       print('The number n is set to be the size of spike trains')
       n = N

    #plot rater
    i = 0
    while i< n:
        if spike_train[i, :].sum()>0.:
          t_sp = range_t[spike_train[i, :]>0.5] #spike times 
          plt.plot(t_sp, i*np.ones(len(t_sp)), 'k|', ms=10, markeredgewidth=2)
        i += 1
    plt.xlim([range_t[0], range_t[-1]])
    plt.ylim([-0.5, n+0.5])    
    plt.xlabel('Time (ms)', fontsize=12)
    plt.ylabel('Neuron ID', fontsize=12);

Execute the cell below to generate a hundred Poisson trains.

In [ ]:
# we can use Poisson_generator mimic presynaptic spike trains
pars = default_pars()
pre_spike_train = Poisson_generator(pars, rate=50, n=100)

my_raster_Poisson(pars['range_t'], pre_spike_train, 100) 

# print(help(Poisson_generator))
# print('\n')
print(help(my_raster_Poisson)) # use `help` to learn about the function

# <font color='salmon'> How do we make sure that the above spike trains are follwoing Poisson statistics?</font>
A Poisson process must have the following properties:
- The ratio of the mean and variance of spike count is 1.
- Inter-spike-invervals are exponentially distributed
- Spike times are irregular i.e. $CV_{ISI} = 1$
- Adjancent spike internal are independent of each other. 

In the following we make one test.
Execute the cell below and check that the interval time between two consecutive Poisson events follows the exponential distribution. 

You should write your own code to check for the other properties. 

In [ ]:
pars = default_pars()
pars['T'] = 1000.
pre_spike_train = Poisson_generator(pars, rate=10, n=5000)
spT = pre_spike_train[pre_spike_train.sum(axis=1)>1.,:]
id_o = np.arange(pars['range_t'].size)
t_isi_temp = id_o[spT[0, :]>0.5]
t_isi = np.diff(t_isi_temp)*pars['dt']
for i in range(spT.shape[0]-1):
    t_isi_temp = id_o[spT[i+1, :]>0.5]
    t_isi = np.concatenate((t_isi, np.diff(t_isi_temp)*pars['dt']))

plt.figure(figsize=(9., 4.))

plt.subplot(1,2,1)    
isi_count, isi_bin = np.histogram(t_isi, bins = np.arange(0, 400.1, 4.))
isi_bin = 0.5 * (isi_bin[1:] + isi_bin[:-1])
plt.bar(isi_bin, isi_count, width=4.0, color='b', alpha=0.5)
#plt.legend(loc='upper right')
plt.xlabel('ISI (ms)')
plt.ylabel('Number')

plt.subplot(1,2,2)
isi_count_normalize = isi_count/isi_count.sum()/(isi_bin[1]-isi_bin[0]) * 1000. #note the units
plt.semilogy(isi_bin/1000., isi_count_normalize, 'b', alpha=0.5)
plt.xlabel('ISI (s)')
plt.ylabel(r'Probability density function (s$^{-1}$)')
plt.tight_layout()

### Simulate LIF neuron with conductance-based synapses

We are now ready to simulate a LIF neuron with conductance based synaptic inputs! The following code defines the LIF neuron with synaptic input modelled as conductance transients.

In [ ]:
def run_LIF_cond(pars, I_inj, pre_spike_train_ex, pre_spike_train_in):
    '''
    conductance-based LIF dynamics
    
    Expects:
    pars               : parameter dictionary
    I_inj              : injected current [pA]. The injected current here can be a value or an array
    pre_spike_train_ex : spike train input from presynaptic excitatory neuron
    pre_spike_train_in : spike train input from presynaptic inhibitory neuron
    
    Returns:
    rec_spikes : spike times
    rec_v      : mebrane potential
    gE         : postsynaptic excitatory conductance
    gI         : postsynaptic inhibitory conductance
    '''
    
    # Retrieve parameters
    V_th, V_reset = pars['V_th'], pars['V_reset']
    tau_m, g_L = pars['tau_m'], pars['g_L']
    V_init, V_L = pars['V_init'], pars['V_L']
    gE_bar, gI_bar = pars['gE_bar'], pars['gI_bar']
    VE, VI = pars['VE'], pars['VI']
    tau_syn_E, tau_syn_I = pars['tau_syn_E'], pars['tau_syn_I']
    tref = pars['tref'] 
    dt, range_t = pars['dt'], pars['range_t']
    Lt = range_t.size
    
    # Initialize
    tr = 0.
    v = np.zeros(Lt)
    v[0] = V_init
    gE = np.zeros(Lt)
    gI = np.zeros(Lt)
    I = I_inj * np.ones(Lt) #ensure I has length Lt

    if pre_spike_train_ex.max() == 0:
      pre_spike_train_ex_total = np.zeros(Lt)
    else:
      pre_spike_train_ex_total = pre_spike_train_ex.sum(axis=0) * np.ones(Lt)

    if pre_spike_train_in.max() == 0:
      pre_spike_train_in_total = np.zeros(Lt)
    else:
      pre_spike_train_in_total = pre_spike_train_in.sum(axis=0) * np.ones(Lt)
    
    # simulation
    rec_spikes = [] # recording spike times
    for it in range(Lt-1):
      if tr >0:
            v[it] = V_reset
            tr = tr-1
      elif v[it] >= V_th:         #reset voltage and record spike event
            rec_spikes.append(it)
            v[it] = V_reset
            tr = tref/dt
      #update the synaptic conductance
      gE[it+1] = gE[it] - (dt/tau_syn_E)*gE[it] + gE_bar*pre_spike_train_ex_total[it+1]
      gI[it+1] = gI[it] - (dt/tau_syn_I)*gI[it] + gI_bar*pre_spike_train_in_total[it+1]
        
      #calculate the increment of the membrane potential
      dv = (-(v[it]-V_L) - (gE[it+1]/g_L)*(v[it]-VE) - \
              (gI[it+1]/g_L)*(v[it]-VI) + I[it]/g_L) * (dt/tau_m)
        
      #update membrane potential
      v[it+1] = v[it] + dv
        
    rec_spikes = np.array(rec_spikes) * dt
        
    return v, rec_spikes, gE, gI

### Excitation and inhibition balance
In vivo a neuron in the neocortex receives a number of excitatory and inhibitory inputs. At the simplest we can assume that these input spikes follow Poissonian statistics. 
In the following we will study how varying the ratio of excitatory and inhibitory inputs changes the firing rate and the spike time regularity. 

To change the exc. and inh. inputs we will vary the firing rates. But if you wish you can vary the strength and/or number of these connections. 

In [ ]:
def ei_balance_isi_regularity(exc_rate = 15, inh_rate = 45):
  pars = default_pars()
  # Set the duration of simulartion
  pars['T'] = 1000.
  pars['range_t'] = np.arange(0, pars['T'], pars['dt']) # Vector of discretized time points [ms]

  # Set the firing rate of the excitatory and inhibitory inputs
  exc_rate = 15. # excitatory firing rate
  inh_rate = 45. # inhibitory firing rate

  gE        = 3.   #nS
  E_rev     = 0.    #mv
  tau_syn_e = 2.    #ms
  gI        = 3.   #nS
  tau_syn_i = 5.   #ms
  I_rev     = -80.  #mv
  V_th      = -55.  #mV

  pars['V_L'] = -70. # mV
  pars['gE_bar']    = gE
  pars['VE']        = E_rev
  pars['tau_syn_E'] = tau_syn_e
  pars['gI_bar']    = gI
  pars['VI']        = I_rev
  pars['tau_syn_I'] = tau_syn_i


  pre_spike_train_ex = Poisson_generator(pars, rate=exc_rate, n=80)
  pre_spike_train_in = Poisson_generator(pars, rate=inh_rate, n=20) # 4:1


  # Lets first simulate a neuron with identical input but with no spike threshold 
  # so that we can look at the free membrane potential
  pars['V_th'] = 1e3

  v_fmp, rec_spikes, gE, gI = run_LIF_cond(pars, 0, pre_spike_train_ex, pre_spike_train_in)

  # Now simulate a LIP with a spike threshold
  pars['V_th'] = -55. 
  v, rec_spikes, gE, gI = run_LIF_cond(pars, 0, pre_spike_train_ex, pre_spike_train_in)

  spike_rate = 1e3*len(rec_spikes)/pars['T']

  cv_isi = 0.
  if len(rec_spikes)>3:
    isi = np.diff(rec_spikes)
    cv_isi = np.std(isi)/np.mean(isi)

  # histogram of the free membrane potential
  ed = np.arange(-80.,-45,0.1)
  fmp_hh, edx = np.histogram(v_fmp,ed)
  fmp_hh = fmp_hh/len(v_fmp)
    
  print('Spike Rate = ', str(spike_rate))
  print('CV ISI = ', str(cv_isi))
  print('Mean of Free Mem Pot = ', str(np.mean(v_fmp)))
  print('STD of Free Mem Pot = ', str(np.std(v_fmp)))


  plt.figure(figsize=(15., 10))

  plt.subplot(2,1,1)
  plt.plot(pars['range_t'], v_fmp, 'r',label = 'Free mem. pot.')
  plt.plot(pars['range_t'], v, label = 'mem. pot with spk thr')
  plt.plot([pars['range_t'][0], pars['range_t'][-1]],[pars['V_th'], pars['V_th'] ],'--',label = 'Spike Threshold') 
  plt.plot([pars['range_t'][0], pars['range_t'][-1]],[np.mean(v_fmp), np.mean(v_fmp) ],'--',label = 'Mean Free Mem. Pot.') 
  plt.xlabel('Time (ms)')
  plt.ylabel('V (mV)');
  plt.legend(loc='upper right')
  plt.plot(fmp_hh*1e4-150,edx[:-1],'r')

  plt.subplot(2,2,3)
  plt.plot(pars['range_t'], gE, 'g')
  plt.xlabel('Time (ms)')
  plt.ylabel(r'$g_E$ (nS)')

  plt.subplot(2,2,4)
  plt.plot(pars['range_t'], gI, 'b')
  plt.xlabel('Time (ms)')
  plt.ylabel(r'$g_I$ (nS)')

  plt.tight_layout()
  return rec_spikes

rec_spikes = widgets.interact(ei_balance_isi_regularity, inh_rate = (10., 60., 5.), exc_rate = (5.,20.,2))

# <font color=salmon>Think</font>
- You might have checked that when you increased Exc rate output firing rate increased and when you increased Inh rate output firing rate decreased. How do you think the output firing rate varies when both exc and inh rates are varied simultaneously?  

- How much you can increase the spike pattern variability? Note we are injected Poisson type spikes, under what condition the neuron may also respond with Poisson type spikes? 


### Effect of excitatory and inhibitory input on the neuron spike output (rate and irregularity)

In the following we will systematically vary the exc. and inh. rate. We will measure the mean free membrane potential, spike rate and CV of the output spike pattern. 

*Mean free membrane potential* is the mem. pot. of the neuron when spike threshold is removed. This is completely artificial but this variable allows us to get an idea of how strong the input it. Especially we are interested in knowing the  mean and std. of the free mem. pot.

Depending on the range and resolution it may take some time so start with a coarser resolution.

In [ ]:
# Set the range of Excitatory and inhibtory firing rates
exc_rate = np.arange(5.,30.,2) # excitatory firing rate [SET YOUR RANGE HERE]
inh_rate = np.arange(5.,80.,5) # inhibitory firing rate [SET YOUR RANGE HERE]

# Set the duration of simulation
gE        = 3.   #nS
E_rev     = 0.    #mv
tau_syn_e = 2.    #ms
gI        = 3.   #nS
tau_syn_i = 5.   #ms
I_rev     = -80.  #mv
V_th      = -55.  #mV

pars['T'] = 1000.
pars['range_t'] = np.arange(0, pars['T'], pars['dt']) # Vector of discretized time points [ms]

pars['V_L']    = -70. # mV
pars['gE_bar']    = gE
pars['VE']        = E_rev
pars['tau_syn_E'] = tau_syn_e
pars['gI_bar']    = gI
pars['VI']        = I_rev
pars['tau_syn_I'] = tau_syn_i

mean_fmp = np.zeros((len(exc_rate),len(inh_rate)))
std_fmp = np.zeros((len(exc_rate),len(inh_rate)))
spk_rate = np.zeros((len(exc_rate),len(inh_rate)))
cv_isi = np.empty((len(exc_rate),len(inh_rate)))
cv_isi[:] = np.NaN

for ii in range(len(exc_rate)):
  for jj in range(len(inh_rate)):
    pre_spike_train_ex = Poisson_generator(pars, rate=exc_rate[ii], n=80)
    pre_spike_train_in = Poisson_generator(pars, rate=inh_rate[jj], n=20) # 4:1
    pars['V_th'] = 1e4
    v_fmp, rec_spikes, gE, gI = run_LIF_cond(pars, 0, pre_spike_train_ex, pre_spike_train_in)
    pars['V_th'] = V_th 
    v, rec_spikes, gE, gI = run_LIF_cond(pars, 0, pre_spike_train_ex, pre_spike_train_in)
    mean_fmp[ii,jj] = np.mean(v_fmp)
    std_fmp[ii,jj] = np.std(v_fmp)
    spk_rate[ii,jj] = 1e3*len(rec_spikes)/pars['T']
    if len(rec_spikes)>3:
      isi = np.diff(rec_spikes)
      cv_isi[ii,jj] = np.std(isi)/np.mean(isi)

In [ ]:
plt.figure()

plt.subplot(2,2,1)
plt.pcolor(exc_rate,inh_rate,mean_fmp.T)
plt.colorbar()
#plt.clim([-70,-55])
plt.xlabel('Exc. Rate (Hz)')
plt.ylabel('Inh. Rate (Hz)')
plt.title('Mean of Free Mem. Pot.')

plt.subplot(2,2,2)
plt.pcolor(exc_rate,inh_rate,std_fmp.T)
plt.colorbar()
plt.xlabel('Exc. Rate (Hz)')
plt.ylabel('Inh. Rate (Hz)')
plt.title('Std. of Free Mem. Pot.')


# Plot Firing rate as a function of both mean and std. Firing rate is rendered as color
plt.subplot(2,2,3)
plt.pcolor(exc_rate,inh_rate,spk_rate.T)
plt.colorbar()
plt.clim(0,50)
plt.xlabel('Exc. Rate (Hz)')
plt.ylabel('Inh. Rate (Hz)')
plt.title('Mean Firing Rate')

# Plot Spike time irregularity (CV_ISI) as a function of both mean and std. CV_isi is rendered as color
plt.subplot(2,2,4)
plt.pcolor(exc_rate,inh_rate,cv_isi.T)
plt.colorbar()
plt.clim(0,1)
plt.xlabel('Exc. Rate (Hz)')
plt.ylabel('Inh. Rate (Hz)')
plt.title('CV of ISI')

plt.tight_layout()

plt.figure()
plt.figure(figsize=(20, 8))
plt.subplot(1,4,1)
plt.plot(mean_fmp,cv_isi,'.b')
plt.plot([V_th, V_th],[0 ,1.4],'r')
plt.ylim((0,1.4))
#plt.xlim((-62,-45))
plt.xlim((-65,-45))
plt.xlabel('Mean Free Mem. Pot. (mV)')
plt.ylabel('CV of ISI')

plt.subplot(1,4,2)
plt.plot(mean_fmp,std_fmp,'.b')
plt.plot([V_th, V_th],[2 ,6],'r')
plt.ylim((2.5,6))
plt.xlim((-65,-45))
plt.ylabel('STD. Free Mem. Pot. (mV)')
plt.xlabel('Mean Free Mem. Pot. (mV)')

plt.subplot(1,4,3)
plt.plot(std_fmp,cv_isi,'.b')
plt.ylim((0,1.4))
#plt.xlim((-62,-45))
plt.xlabel('STD. Free Mem. Pot. (mV)')
plt.ylabel('CV of ISI')


plt.subplot(1,4,4)
plt.plot(spk_rate,cv_isi,'.b')
plt.ylim((0,1.4))
plt.xlabel('Output spike rate (Hz)')
plt.ylabel('CV of ISI')

plt.tight_layout()


# <font color=salmon>Try to answer the following</font>
- What are input configurations that result in irregular spiking outpout?
- One of the definition of excitation and inhibition balance is that mean free membrane potential remains constant as exc and inh input rates are increased. What do you think happens to the neuron firing rate as we change exc. and inh. rates while keeping the neuron in balance. 


### Mean-driven and Fluctuation-driven regimes

If we look at the figure above, we note that when the mean free mem. pot. is above the spike threshold, the fluctuations in the free mem pot are rather small and the neuron spikes in more or less regular fashion. This regime when mean of the free mem. pot. is above the spike threshold is called the **Mean driven Regime**. 


When the mean free mem. pot. is below the spike threshold, the fluctuations in the free mem pot are big and the neuron spikes are driven by the fluctuations. As a consequence the neuron spikes in more or less Poisson fashion. This regime when mean of the free mem. pot. is below the spike threshold and spikes are driven by fluctuation is called the **Fluctuation driven Regime**. 